In [1]:
import pandas as pd 
import os
import csv 
import numpy as np 

In [2]:
train_path = r"C:\Users\Farz Jamal\Downloads\AI-Sentiment-Analysis-on-IMDB-Dataset-master\aclImdb\train\\" # source data
test_path = r"C:\Users\Farz Jamal\Downloads\AI-Sentiment-Analysis-on-IMDB-Dataset-master\test\imdb_te.csv" # test data  

In [3]:

'''
IMDB_DATA_PREPROCESS explores the neg and pos folders from aclImdb/train and creates a output_file in the required format
Inpath - Path of the training samples 
Outpath - Path were the file has to be saved 
Name  - Name with which the file has to be saved 
Mix - Used for shuffling the data 
'''
def imdb_data_preprocess(inpath, outpath="./", name="imdb_training.csv", mix=False):


    stopwords = open(r"C:\Users\Farz Jamal\Downloads\AI-Sentiment-Analysis-on-IMDB-Dataset-master\stopwords.en.txt", 'r' , encoding="ISO-8859-1").read()
    stopwords = stopwords.split("\n")   #stopwords is a list of Stop Words

    indices,text,rating = [],[],[]

    i =  0 

    for filename in os.listdir(inpath+"pos"):
        data = open(inpath+"pos/"+filename, 'r' , encoding="ISO-8859-1").read()
        data = remove_stopwords(data, stopwords)
        indices.append(i)
        text.append(data)
        rating.append("1")
        i+=1

    for filename in os.listdir(inpath+"neg"):
        data = open(inpath+"neg/"+filename, 'r' , encoding="ISO-8859-1").read()
        data = remove_stopwords(data, stopwords)
        indices.append(i)
        text.append(data)
        rating.append("0")
        i+=1

    Dataset = list(zip(indices,text,rating))   #Each element of this list will be a tuple of the form (index,text,rating)

    if mix:
        np.random.shuffle(Dataset)    #shuffles the dataset

    df = pd.DataFrame(data = Dataset, columns=['row_Number', 'text', 'polarity'])
    df.to_csv(outpath+name, index=False, header=True)

    pass

In [4]:

'''
REMOVE_STOPWORDS takes a sentence and a list of stopwords as inputs.Then returns the sentence without any stopwords. 
Sentence - The input from which the stopwords have to be removed
Stopwords - A list of stopwords  
'''
def remove_stopwords(sentence, stopwords):
    sentencewords = sentence.split()     #splits words of sentence into a list
    resultwords  = [word for word in sentencewords if word.lower() not in stopwords]
    result = ' '.join(resultwords)
    return result


'''
UNIGRAM_PROCESS takes the data to be fit as the input and returns a vectorizer of the unigram as output 
Data - The data for which the unigram model has to be fit 
'''
def unigram_process(data):
    from sklearn.feature_extraction.text import CountVectorizer
    vectorizer = CountVectorizer()
    vectorizer = vectorizer.fit(data)
    return vectorizer


'''
BIGRAM_PROCESS takes the data to be fit as the input and returns a vectorizer of the bigram as output 
Data - The data for which the bigram model has to be fit 
'''
def bigram_process(data):
    from sklearn.feature_extraction.text import CountVectorizer
    vectorizer = CountVectorizer(ngram_range=(1,2))    #preserves one word as well as two word sequence thanks to the ngram_range parameter
    vectorizer = vectorizer.fit(data)
    return vectorizer


'''
TFIDF_PROCESS takes the data to be fit as the input and returns a vectorizer of the tfidf as output 
Data - The data for which the bigram model has to be fit 
'''
def tfidf_process(data):
    from sklearn.feature_extraction.text import TfidfTransformer 
    transformer = TfidfTransformer()
    transformer = transformer.fit(data)
    return transformer




In [5]:
'''
RETRIEVE_DATA takes a CSV file as the input and returns the corresponding arrays of labels and data as output. 
Name - Name of the csv file 
Train - If train is True, both the data and labels are returned. Else only the data is returned 
'''
def retrieve_data(name="imdb_training.csv", train=True):
    import pandas as pd 
    data = pd.read_csv(name,header=0, encoding = 'ISO-8859-1')
    X = data['text']

    if train:
        Y = data['polarity']
        return X, Y

    return X

'''
STOCHASTIC_DESCENT applies Stochastic on the training data and returns the predicted labels 
Xtrain - Training Data
Ytrain - Training Labels
Xtest - Test Data 
'''
def stochastic_descent(Xtrain, Ytrain, Xtest):
    from sklearn.linear_model import SGDClassifier 
    clf = SGDClassifier(loss="hinge", penalty="l1", n_iter=25)        #hinge loss , therefore using a SVM
    print ("Fitting SGD")
    clf.fit(Xtrain, Ytrain)
    print ("Predicting using SGD")
    Ytest = clf.predict(Xtest)
    return Ytest


'''
ACCURACY finds the accuracy in percentage given the training and test labels 
Ytrain - One set of labels 
Ytest - Other set of labels 
'''
def accuracy(Ytrain, Ytest):
    assert (len(Ytrain)==len(Ytest))
    num =  sum([1 for i, word in enumerate(Ytrain) if Ytest[i]==word])
    n = len(Ytrain)  
    return (num*100)/n


'''
WRITE_TXT writes the given data to a text file 
Data - Data to be written to the text file 
Name - Name of the file 
'''
def write_txt(data, name):
    data = ''.join(str(word) for word in data)
    file = open(name, 'w')
    file.write(data)
    file.close()
    pass 


In [6]:
if __name__ == "__main__":
    import time
    start = time.time()
    print ("Preprocessing the training_data--")
    imdb_data_preprocess(inpath=train_path, mix=True)
    print ("Done with preprocessing. Now, will retreieve the training data in the required format")
    [Xtrain_text, Ytrain] = retrieve_data()
    print ("Retrieved the training data. Now will retrieve the test data in the required format")
    Xtest_text = retrieve_data(name=test_path, train=False)
    print ("Retrieved the test data. Now will initialize the model \n\n")


    print ("-----------------------ANALYSIS ON THE TRAINING DATA---------------------------")
    uni_vectorizer = unigram_process(Xtrain_text)
    print ("Fitting the unigram model")
    Xtrain_uni = uni_vectorizer.transform(Xtrain_text)
    print ("After fitting ")
    print ("\n")

    bi_vectorizer = bigram_process(Xtrain_text)
    print ("Fitting the bigram model")
    Xtrain_bi = bi_vectorizer.transform(Xtrain_text)
    print ("After fitting ")
    print ("Applying the stochastic descent")
    Y_bi = stochastic_descent(Xtrain_bi, Ytrain, Xtrain_bi)
    print ("Done with  stochastic descent")
    print ("Accuracy for the Bigram Model on the training data ", accuracy(Ytrain, Y_bi))
    print ("\n")

    uni_tfidf_transformer = tfidf_process(Xtrain_uni)
    print ("Fitting the tfidf for unigram model")
    Xtrain_tf_uni = uni_tfidf_transformer.transform(Xtrain_uni)
    print ("After fitting TFIDF")
    print ("Applying the stochastic descent")
    Y_tf_uni = stochastic_descent(Xtrain_tf_uni, Ytrain, Xtrain_tf_uni)
    print ("Done with  stochastic descent")
    print ("Accuracy for the Unigram TFIDF Model is ", accuracy(Ytrain, Y_tf_uni))
    print ("\n")


    bi_tfidf_transformer = tfidf_process(Xtrain_bi)
    print ("Fitting the tfidf for bigram model")
    Xtrain_tf_bi = bi_tfidf_transformer.transform(Xtrain_bi)
    print ("After fitting TFIDF")
    print ("Applying the stochastic descent")
    Y_tf_bi = stochastic_descent(Xtrain_tf_bi, Ytrain, Xtrain_tf_bi)
    print ("Done with  stochastic descent")
    print ("Accuracy for the Unigram TFIDF Model is ", accuracy(Ytrain, Y_tf_bi))
    print ("\n")


    print ("-----------------------ANALYSIS ON THE TEST DATA ---------------------------")
    print ("Unigram Model on the Test Data--")
    Xtest_uni = uni_vectorizer.transform(Xtest_text)
    print ("Applying the stochastic descent")
    Ytest_uni = stochastic_descent(Xtrain_uni, Ytrain, Xtest_uni)
    write_txt(Ytest_uni, name="unigram.output.txt")
    print ("Done with  stochastic descent")
    print ("\n")


    print ("Bigram Model on the Test Data--")
    Xtest_bi = bi_vectorizer.transform(Xtest_text)
    print ("Applying the stochastic descent")
    Ytest_bi = stochastic_descent(Xtrain_bi, Ytrain, Xtest_bi)
    write_txt(Ytest_bi, name="bigram.output.txt")
    print ("Done with  stochastic descent")
    print ("\n")

    print ("Unigram TF Model on the Test Data--")
    Xtest_tf_uni = uni_tfidf_transformer.transform(Xtest_uni)
    print ("Applying the stochastic descent")
    Ytest_tf_uni = stochastic_descent(Xtrain_tf_uni, Ytrain, Xtest_tf_uni)
    write_txt(Ytest_tf_uni, name="unigramtfidf.output.txt")
    print ("Done with  stochastic descent")
    print ("\n")

    print ("Bigram TF Model on the Test Data--")
    Xtest_tf_bi = bi_tfidf_transformer.transform(Xtest_bi)
    print ("Applying the stochastic descent")
    Ytest_tf_bi = stochastic_descent(Xtrain_tf_bi, Ytrain, Xtest_tf_bi)
    write_txt(Ytest_tf_bi, name="bigramtfidf.output.txt")
    print ("Done with  stochastic descent")
    print ("\n")
    
    print ("Total time taken = ", time.time()-start, " seconds")
    pass

Preprocessing the training_data--
Done with preprocessing. Now, will retreieve the training data in the required format
Retrieved the training data. Now will retrieve the test data in the required format
Retrieved the test data. Now will initialize the model 


-----------------------ANALYSIS ON THE TRAINING DATA---------------------------
Fitting the unigram model
After fitting 


Fitting the bigram model
After fitting 
Applying the stochastic descent
Fitting SGD


G:\anaconda\envs\tensorflow\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


Predicting using SGD
Done with  stochastic descent
Accuracy for the Bigram Model on the training data  94.052


Fitting the tfidf for unigram model
After fitting TFIDF
Applying the stochastic descent
Fitting SGD


G:\anaconda\envs\tensorflow\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


Predicting using SGD
Done with  stochastic descent
Accuracy for the Unigram TFIDF Model is  88.792


Fitting the tfidf for bigram model
After fitting TFIDF
Applying the stochastic descent
Fitting SGD


...

Predicting using SGD
Done with  stochastic descent


Unigram TF Model on the Test Data--
Applying the stochastic descent
Fitting SGD


G:\anaconda\envs\tensorflow\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


Predicting using SGD
Done with  stochastic descent


Bigram TF Model on the Test Data--
Applying the stochastic descent
Fitting SGD


G:\anaconda\envs\tensorflow\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


Predicting using SGD
Done with  stochastic descent


Total time taken =  312.6678087711334  seconds
